## This code illustrates how to get S&P500 index constituents and their identifiers from CRSP and Compustat
- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [3]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds

### Step 1: Connect to WRDS

In [4]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

Enter your WRDS username [mitunl]:mitun179
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [14]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date <= a.ending
                        and b.date = '12/30/2022'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])


In [15]:
sp500

,permno,start,ending,date,ret
0,10104.0,1989-08-03,2022-12-30,2022-12-30,-0.015537
1,10107.0,1994-06-07,2022-12-30,2022-12-30,-0.060045
2,10138.0,1999-10-13,2022-12-30,2022-12-30,-0.117284
3,10145.0,1925-12-31,2022-12-30,2022-12-30,-0.023913
4,10516.0,1981-07-30,2022-12-30,2022-12-30,-0.047692
...,...,...,...,...,...
498,93096.0,2012-12-03,2022-12-30,2022-12-30,-0.034731
499,93132.0,2018-10-11,2022-12-30,2022-12-30,-0.080324
500,93246.0,2021-03-22,2022-12-30,2022-12-30,-0.046058
501,93429.0,2017-03-01,2022-12-30,2022-12-30,-0.010801


### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- You don't need this step if only PERMNO is required
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [16]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

In [17]:
mse

,comnam,ncusip,namedt,nameendt,permno,shrcd,exchcd,hsiccd,ticker
0,OPTIMUM MANUFACTURING INC,68391610,1986-01-07,1986-12-03,10000.0,10.0,3.0,3990.0,OMFGA
1,OPTIMUM MANUFACTURING INC,68391610,1986-12-04,1987-03-09,10000.0,10.0,3.0,3990.0,OMFGA
2,OPTIMUM MANUFACTURING INC,68391610,1987-03-10,1987-06-11,10000.0,10.0,3.0,3990.0,OMFGA
3,GREAT FALLS GAS CO,39040610,1986-01-09,1993-11-21,10001.0,11.0,3.0,4925.0,GFGC
4,ENERGY WEST INC,29274A10,1993-11-22,2004-06-09,10001.0,11.0,3.0,4925.0,EWST
...,...,...,...,...,...,...,...,...,...
111618,VOLTARI CORP,92870X30,2013-04-24,2016-12-22,93433.0,11.0,3.0,9999.0,VLTC
111619,S & W SEED CO,78513510,2010-06-14,2022-12-30,93434.0,11.0,3.0,9999.0,SANW
111620,SINO CLEAN ENERGY INC,82936G20,2010-06-14,2012-05-18,93435.0,11.0,3.0,6163.0,SCEI
111621,TESLA MOTORS INC,88160R10,2010-06-29,2017-02-01,93436.0,11.0,3.0,9999.0,TSLA


In [19]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full['ticker']

3       ORCL
5       MSFT
11      TROW
30       HON
39       ADM
        ... 
3464      DG
3466    FTNT
3467    GNRC
3471    CBOE
3473    TSLA
Name: ticker, Length: 503, dtype: object

In [21]:
ticker_list = sp500_full['ticker']

# Convert the list to a DataFrame
ticker_df = pd.DataFrame({'ticker': ticker_list})

# Reset the index
ticker_df.reset_index(drop=True, inplace=True)
ticker_df

,ticker
0,ORCL
1,MSFT
2,TROW
3,HON
4,ADM
...,...
498,DG
499,FTNT
500,GNRC
501,CBOE


In [22]:
file_path = "ticker_data.csv"  # Replace with your desired file path

# Save the DataFrame to a CSV file
ticker_df.to_csv(file_path, index=False) 